# Demonstration of basic PET capabilities with SIRF: IO and projections
This demonstration shows how to read images and data, display them. It then
illustrates how to use an AcquisitionModel to forward and backproject.

This notebook is largely superseded by the [Introductory/ notebooks](../Introductory/), which illustrate the same concepts with PET, MR and x-ray CT, but kept here as there are a few differences:
- It uses similar images but smaller for speed.
- It gives a little bit more information on dimensions of `AcquisitionData`.
- For display, it uses the SIRF `show` method and also tells you how to run an animation in Python.

This demo is a jupyter notebook, i.e. intended to be run step by step.
You could export it as a Python file and run it one go, but that might
make little sense as the figures are not labelled.

Author: Kris Thielemans  
First version: 8th of September 2016  
Second version: 17th of May 2018  
Third version: June 2021

CCP SyneRBI Synergistic Image Reconstruction Framework (SIRF).  
Copyright 2015 - 2017 Rutherford Appleton Laboratory STFC.  
Copyright 2015 - 2018, 2021 University College London.

This is software developed for the Collaborative Computational
Project in Synergistic Reconstruction for Biomedical Imaging
(http://www.ccpsynerbi.ac.uk/).

SPDX-License-Identifier: Apache-2.0

# Initial set-up

make sure figures appear inline and animations work

In [ ]:
%matplotlib widget

If the above failed, or you experience problems plotting, you should restart the Python kernel and use the following line instead.

In [ ]:
%matplotlib inline

In [ ]:
import notebook_setup

#%% Initial imports etc
import numpy
from numpy.linalg import norm
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import os
import sys
import shutil

### Use the 'pet' prefix for all STIR-based SIRF functions
This is done here to explicitly differentiate between SIRF pet functions and anything else.

In [ ]:
import sirf.STIR as pet
from sirf.Utilities import examples_data_path

### redirect STIR messages to some files
STIR output can be a bit verbose. There are currently also some spurious warnings. To avoid distracting you, we redirect these messages to some files. You can check these if things go wrong.


In [ ]:
msg_red = pet.MessageRedirector('info.txt', 'warnings.txt')

### Set up the data and working directory for the notebook.

Please make sure that you have run the `download_data.sh` script first. See the [Introductory/introduction notebook](../Introductory/introduction.ipynb) for more information.

In [ ]:
from sirf_exercises import exercises_data_path
from sirf_exercises import cd_to_working_dir
cd_to_working_dir('PET', 'image_creation_and_simulation')

Copy files to a working folder and change directory to where these files are
We do this to avoid cluttering your SIRF files. This way, you can delete 
`working_folder` and start from scratch.

In [ ]:
# Find directory with input files
org_data_dir=os.path.join(examples_data_path('PET'),'brain')

In [ ]:
# remove existing files just to be sure
shutil.rmtree('./brain',True)
# copy the whole folder
shutil.copytree(org_data_dir,'./brain')
# and go there
os.chdir('./brain')

OK. finally done with initial set-up...

# Basic image manipulations
Images (like most other things in SIRF) are represented as *objects*, in this case of type `ImageData`.
In practice, this means that you can only manipulate its data via *methods*.

Image objects contain the actual voxel values, but also information on the number of voxels,
voxel size, etc. There are methods to get this information.

There are additional methods for other manipulations, such as basic image arithmetic (e.g.,
you can add image objects).

## Read in images
Here we will read some images provided with the demo using the `ImageData` class.
These are in Interfile format. (A text header called `something.hv` pointing to a `.v` file with the binary data).

In [ ]:
image = pet.ImageData('emission.hv')
mu_map = pet.ImageData('attenuation.hv')

## What is an ImageData object?
Images are represented by objects with several methods. Probably the most important method 
is `as_array()` which we'll use below.

Let's see what all the methods are.

In [ ]:
help(pet.ImageData)

In [ ]:
#%% Use as_array to extract an array of voxel values
# The resulting array as a `numpy` array, as standard in Python.
image_array=image.as_array()
# We can use the standard `numpy` methods on this array, such as getting its `shape` (i.e. dimensions).
print(image_array.shape)
# Whenever we want to do something with the image-values, we have to do it via this array.
# Let's print a voxel-value.
print(image_array[10,40,60])

In [ ]:
#%% Manipulate the image data for illustration
# Multiply the data with a factor
image_array *= 0.01
# Stick this new data into the original image object.
# (This will not modify the file content, only the variable in memory.)
image.fill(image_array)
print(image_array[10,40,60])

In [ ]:
#%% You can do basic math manipulations with ImageData objects 
# So the above lines can be done directly on the `image` object
image *= 0.01
# Let's check
image_array=image.as_array()
print(image_array[10,40,60])

In [ ]:
#%% Display the middle slice of the image (which is really a 3D volume)

# Get the middle slice number
slice_num = image_array.shape[0]//2
# Display the slice using the `ImageData.show` method
image.show(slice_num, title='emission image')

Note that when using the `%matplotlib widget` or `inline` back-ends, the `title` is currently not shown when using `show`. Apologies

In [ ]:
#%% Some other things to do with ImageData objects
print(image.voxel_sizes())
another_image=image.clone()
an_image_with_fixed_values = image.get_uniform_copy(5)

# Forward and back projection
Now we will do some PET projections!
SIRF uses AcquisitionModel as the object to do forward and back-projections.
We will create an AcquisitionModel object and then use it to forward project
our image etc.

## Create a SIRF acquisition model
We will use the ray-tracing matrix here as our simple PET model.
There is more to the accquisition model, but that's for another demo.

In [ ]:
am = pet.AcquisitionModelUsingRayTracingMatrix()

We need to say what scanner to use, what dimensions etc.
The easiest way to do this by using existing PET data as a 'template'. 
Here, we read a file supplied with the demo as an `AcquisitionData` object.

In [ ]:
templ = pet.AcquisitionData('template_sinogram.hs')
# Now set-up our acquisition model with all information that it needs about the data and image.
am.set_up(templ,image); 

*Warning*: the provided template has no actual data, and is essentially only a header. Therefore, most operations using the `templ` variable would fail. However, it is sufficient for getting geometry etc.

The `AcquisitionModel` is now ready for use.

## Do a forward projection of our image
'forward projection' is the terminology used in PET to simulate the acquisition.
Input is a SIRF `ImageData` object (not image_array), output is an `AcquisitionData` object.

In [ ]:
acquired_data=am.forward(image)

Check what methods an `AcquisitionData` object has

In [ ]:
help(acquired_data)

AcquisitionData are 4D, organised by time-of-flight (TOF), (2D) sinograms, views and radial positions. Note that there is no TOF in this template, so the first dimension has size 1.

(In fact, SIRF 3.4 does not provide TOF functionality yet, but this makes sure that dimensions will of PET `AcquisitionData` will remain 4D once we have added TOF).

In [ ]:
#%% Let's get the Python array
acquisition_array = acquired_data.as_array()
print(acquisition_array.shape)

As you see, this data is very small. That means that future exercises will be fast, but also that resolution is going to be bad...

In [ ]:
#%% Display bitmap of the middle sinogram
# The sinogram index is the second dimension of an array (but Python counts from 0...)
sino_num = acquisition_array.shape[1]//2
acquired_data.show(sino_num,title='Forward projection');

Display some different 'views' in a movie.

If the animation doesn't work, you might have to change your "backend", e.g. using the `%matplotlib` command.

In [ ]:
bitmaps=[]
fig=plt.figure()
# views are the third index in the data
num_views=acquisition_array.shape[2]
# first construct all the plots
for view in range(0,num_views,4):
    bitmap=plt.imshow(acquisition_array[0,:,view,:,])
    plt.clim(0,acquisition_array.max())
    plt.axis('off')
    bitmaps.append([bitmap])
# Display as animation
ani = animation.ArtistAnimation(fig, bitmaps, interval=100, blit=True, repeat_delay=1000);

You might recognise projections of the bottom half of the brain in that animation (with the bottom displayed at the top!)

## Backprojection
Backprojection multiplies projectino data with the transpose of the forward-projection matrix to go from `AcquisitionData` to an `ImageData`.

In [ ]:
backprojected = am.backward(acquired_data)
backprojected.show(slice_num, title='backprojection');

# What now?

You could have a look at the files that were generated or copied, in particular the `.hs` and `.hv` headers to see if you can make (a little bit of) sense of them. Some of the sizes etc above should be in those headers of course. You can do this by using `File>Open` menu of the Jupyter interface and navigating to the current working directory. Let's print it out again as a reminder.

In [ ]:
%pwd

You could try and repeat this with the `brainweb` data, using some of the code of the "Introductory" notebooks, and see if you can make sense of the animations, sizes etc.

This could go as follows:
  1. create a template for a full mMR acquisition
   ```
   templ = pet.AcquisitionData(os.path.join(examples_data_path('PET'), 'mMR','mMR_template_span11.hs'))
   ```
  2. create im_pet as in the introduction notebook

  3. create acquisition model and use it as above
  
Once done, have a look at the [Synergistic/BrainWeb notebook](../Synergistic/BrainWeb.ipynb) for a complete example.

We suggest you then try the [image_creation_and_simulation notebook](image_creation_and_simulation.ipynb), which constructs images with geometrical objects.